In [19]:
from keras.datasets import mnist
import numpy as np


# desired architecture: 
#     3 layer nn (300,150)
#     softmax activation layer
#     cross entropy loss (gradients will have to be accordingly made to be different for each output (jacobian))

In [32]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()
#train_y = np.reshape(train_y,(train_y.shape[0],1))
#test_y = np.reshape(test_y,(test_y.shape[0],1))

print("train_x shape: {}".format(train_x.shape))
print("train_y shape: {}".format(train_y.shape))
print("test_x shape: {}".format(test_x.shape))
print("test_y shape: {}".format(test_y.shape))

train_x shape: (60000, 28, 28)
train_y shape: (60000,)
test_x shape: (10000, 28, 28)
test_y shape: (10000,)


In [66]:

def one_hot_encode(digit: int) -> np.array:
    encoded = np.zeros(10)
    encoded[digit] = 1
    return encoded


def format_X(X: np.array) -> np.array:
    #assumes X is in the format (m x n0 x n1) where m is num samples, and n0 and n1 have to be collapsed into one frame
    return np.reshape(X,(X.shape[0],X.shape[1]*X.shape[2])).T

def format_Y(Y: np.array) -> np.array:
    return np.array([one_hot_encode(y) for y in Y])

class Net: 
    #input size can be variable, output layer will be a softmax of size given by train_Y
    # X needs to be of the shape n x m, where m is num samples, n is num inputs
    # Y needs to be of the shape o x m, where m is num samples, o is number of neurons in softmax layer
    def __init__(self,layer_dims,train_X,train_Y):
        self.weights = [np.random.randn(layer_dims[0],train_X)]
        self.biases = [np.zeros(np.zeros((l_d,1))) for l_d in layer_dims]
        for idx,l_d in enumerate(layer_dims[1:],1):
            self.weights.append(np.random.randn(l_d,))
    
    @staticmethod
    def sigmoid(z):
        return 1/(1+np.exp(z))
    
    @staticmethod
    def softmax(z):
        return z/np.sum(z)
    
    
        


In [65]:
formatted_X = format_X(train_x)
formatted_Y = format_Y(train_y)
print(formatted_X.shape)
print(formatted_Y.shape)

(784, 60000)
(60000, 10)


In [85]:
np.zeros((10))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])